In [1]:
import torch
print("GPU is availble : -> " , torch.cuda.is_available())

GPU is availble : ->  True


In [2]:
from transformers import pipeline
import pandas as pd



In [13]:
# Pipeline ko correct spelling se recreate karo
# Load English to Urdu translator
translator = pipeline( "translation",
                      model="abdulwaheed1/english-to-urdu-translation-mbart" ,
                       device = 0 )# English → Urdu ONLY )

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [14]:
# first data clean like remove symble which  not a text or also remove the space

def clean_data(text):
 try:
  if pd.isna(text) or text.strip() == '':
    return ''
  result = translator(str(text))
  return result
 except Exception as e:
  print(f"some thing erong {e}")
  return e

In [15]:
df = pd.read_csv("/content/en_test_with_sarcasm.csv")

In [16]:
df.columns


Index(['text', 'binary', 'multiclass'], dtype='object')

In [17]:
len(df)

315

In [18]:
file_name = "/content/en_test_with_sarcasm.csv"
text = "text"
dev_sarcasm_english_urdu = "test_sarcasm_english_urdu.csv"

In [19]:
translate_into_urdu = []
total_text = len(df)
batch_size = 50

for i in range(0, total_text, batch_size):
    batch_end = min(i + batch_size, total_text)
    batch_text = df["text"].iloc[i:batch_end].tolist()

    batch_result_urdu = translator(
        batch_text,
        src_lang="en_XX",    # English source
        tgt_lang="ur_PK"     # ✅ Correct Urdu code
    )

    print(f"processing ......{i + 1} to .....{batch_end}")

    translated_texts = [result['translation_text'] for result in batch_result_urdu]
    translate_into_urdu.extend(translated_texts)

    progress = ((i + batch_size) / total_text) * 100
    print(f"✅ Progress: {min(progress, 100):.1f}% complete")

Your input_length: 201 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 210 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 189 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


processing ......1 to .....50
✅ Progress: 15.9% complete


Your input_length: 200 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


processing ......51 to .....100
✅ Progress: 31.7% complete
processing ......101 to .....150
✅ Progress: 47.6% complete


Your input_length: 206 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


processing ......151 to .....200
✅ Progress: 63.5% complete
processing ......201 to .....250
✅ Progress: 79.4% complete


Your input_length: 184 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 236 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 192 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


processing ......251 to .....300
✅ Progress: 95.2% complete
processing ......301 to .....315
✅ Progress: 100.0% complete


In [20]:
df["translate_urdu"] = translate_into_urdu
file_save = ["binary" , "multiclass" , "translate_urdu"]
print(f"column save{file_save}")

column save['binary', 'multiclass', 'translate_urdu']


In [21]:

final_csv_file = df[file_save].copy()  # Add ()
final_csv_file.to_csv(dev_sarcasm_english_urdu, index=False)  # Fix False

In [22]:

print(f"📁 Output saved as: {dev_sarcasm_english_urdu}")

📁 Output saved as: test_sarcasm_english_urdu.csv
